https://blog.csdn.net/luanpeng825485697/article/details/81048468

In [1]:
from thrift.transport import TSocket,TTransport
from thrift.protocol import TBinaryProtocol
from hbase import Hbase

In [2]:
# thrift默认端口是9090
socket = TSocket.TSocket('h1',9090)
socket.setTimeout(50000)

transport = TTransport.TBufferedTransport(socket)
protocol = TBinaryProtocol.TBinaryProtocol(transport)

client = Hbase.Client(protocol)
socket.open()

print(client.getTableNames())  # 获取当前所有的表名

['mysql_hbase', 'mytable', 'mytable2', 'oracle_hbase', 'table2', 'tablesplit', 'tbl_1', 'tbl_2', 'test4']


In [4]:
from hbase.ttypes import ColumnDescriptor

# 定义列族
column = ColumnDescriptor(name='cf')
# 创建表
client.createTable('test5',[column])

In [5]:
print(client.getTableNames())  # 获取当前所有的表名

['mytable', 'mytable2', 'table2', 'tablesplit', 'tbl_1', 'tbl_2', 'test4', 'test5']


In [6]:
# 禁用表，若表之前未被启用将会引发IOError错误
client.disableTable('test5')

In [7]:
client.isTableEnabled('test5')

False

In [8]:
# 启用表，若表之前未被禁用将会引发IOError错误
client.enableTable('test5')

In [9]:
client.isTableEnabled('test5')

True

In [10]:
client.getTableNames()

['mytable',
 'mytable2',
 'table2',
 'tablesplit',
 'tbl_1',
 'tbl_2',
 'test4',
 'test5']

In [12]:
client.getColumnDescriptors('test5')#获取所有列族信息，返回一个字典 

{'cf:': ColumnDescriptor(name='cf:', maxVersions=3, compression='NONE', inMemory=False, bloomFilterType='NONE', bloomFilterVectorSize=0, bloomFilterNbHashes=0, blockCacheEnabled=False, timeToLive=2147483647)}

In [13]:
client.getTableRegions('test')#获取所有与表关联的regions，返回一个TRegionInfo对象列表 

[]

In [15]:
# 表不存在将会引发IOError(message='java.io.IOException: table does not exist...)错误
# 表未被禁用将会引发IOError(message='org.apache.hadoop.hbase.TableNotDisabledException:...)错误
client.disableTable('test5')
client.deleteTable('test5')

In [16]:
client.getTableNames()

['mytable', 'mytable2', 'table2', 'tablesplit', 'tbl_1', 'tbl_2', 'test4']

In [22]:
result = client.get('table2','rk001','info:name')       # 为一个列表，其中只有一个hbase.ttypes.TCell对象的数据
print (result[0].timestamp)
print (result[0].value)
print(result)

1532504076816
zhangsan
[TCell(value='zhangsan', timestamp=1532504076816)]


In [28]:
result = client.getVer('table2','rk001','info:name',2)       # 为一个列表，其中只有一个hbase.ttypes.TCell对象的数据
print(result)

[TCell(value='zhangsan', timestamp=1532504076816)]


In [29]:
result = client.getVer('table2','rk001','info:name',3)       # 为一个列表，其中只有一个hbase.ttypes.TCell对象的数据
print(result)

[TCell(value='zhangsan', timestamp=1532504076816)]


In [31]:
#获取小于当前时间戳的数据列表
result = client.getVerTs('table2','rk001','info:name',1532504076817,2)       # 为一个列表，其中只有一个hbase.ttypes.TCell对象的数据
print(result)

[TCell(value='zhangsan', timestamp=1532504076816)]


In [34]:
result = client.getRow('table2','rk001')  
result

[TRowResult(row='rk001', columns={'info:name': TCell(value='zhangsan', timestamp=1532504076816)})]

In [36]:
result = client.getRowWithColumns('table2','rk001',['info:name'])
result

[TRowResult(row='rk001', columns={'info:name': TCell(value='zhangsan', timestamp=1532504076816)})]

In [39]:
result = client.getRowTs('table2','rk001',1532504076817)       # 为一个列表，其中只有一个hbase.ttypes.TCell对象的数据
print(result)

[TRowResult(row='rk001', columns={'info:name': TCell(value='zhangsan', timestamp=1532504076816)})]


In [40]:
result = client.getRowWithColumnsTs('table2','rk001',['info:name'],1532504076817)
result

[TRowResult(row='rk001', columns={'info:name': TCell(value='zhangsan', timestamp=1532504076816)})]

In [44]:
from hbase.ttypes import Mutation

mutation = Mutation(column='info:name',value='zhangsan5')

# 插入数据。如果在test表中row行cf:a列存在，将覆盖
client.mutateRow('table2','rk005',[mutation])

In [47]:
from hbase.ttypes import Mutation
# value必须为字符串格式，否则将报错
mutation = Mutation(column='info:name',value='zhangsan6')
client.mutateRowTs('table2','rk006',[mutation],1532504076817)

In [51]:
from hbase.ttypes import Mutation,BatchMutation
mutation1 = Mutation(column='info:name',value='zhangsan7')
mutation2 = Mutation(column='info:name',value='zhangsan71')
batchMutation = BatchMutation('rk007',[mutation1,mutation2])
client.mutateRows('table2',[batchMutation])

In [52]:
from hbase.ttypes import Mutation,BatchMutation
mutation1 = Mutation(column='info:name',value='zhangsan8')
mutation2 = Mutation(column='info:name',value='zhangsan81')
batchMutation = BatchMutation('rk008',[mutation1,mutation2])
client.mutateRowsTs('table2',[batchMutation],1532504076817)

In [55]:
client.deleteAllRow('table2','rk006')

In [59]:
scannerId = client.scannerOpen('table2','rk002',['info:name'])
scannerId

1